In [1]:
"""
This notebook plots predicted and observed tracks for the rs8050061 diQTL (Fig. 3D)
"""

'\nThis notebook plots predicted and observed tracks for the rs8050061 diQTL (Fig. 3D)\n'

In [1]:
import numpy as np
import pyfastx
import os
import h5py
import sys
sys.path.append("../")
from utils import plot_side

In [2]:
# PRINT COMMANDS TO DOWNLOAD AND EXTRACT DATA

# Set SCRATCH to where you want to download data to
SCRATCH = "/Users/adamhe/github/scratch"

URL = "https://zenodo.org/records/10597358/files"
TAR = "example_tracks_and_deepshap.tar.gz"
print(f"wget {URL}/{TAR} -P {SCRATCH}")
print(f"tar -xvzf {SCRATCH}/{TAR} -C {SCRATCH}")

wget https://zenodo.org/records/10597358/files/example_tracks_and_deepshap.tar.gz -P /Users/adamhe/github/scratch
tar -xvzf /Users/adamhe/github/scratch/example_tracks_and_deepshap.tar.gz -C /Users/adamhe/github/scratch


In [32]:
# Load data

# Experimental
y = np.load(
    os.path.join(SCRATCH, "example_tracks_and_deepshap/concat_procap.npz")
)["arr_0"][:, np.r_[250:750, 1250:1750]]

# Predicted
prediction = h5py.File(os.path.join(SCRATCH, "example_tracks_and_deepshap/fold_7_examples_prediction.h5"))
tracks = prediction["track"]
quantity = prediction["quantity"]
quantity = 10 ** (np.hstack(
    [np.log10(quantity[:]), np.ones(quantity.shape[0]).reshape(-1, 1)]
) @ np.array([[1.59687745, -0.80203685]]).T)
y_norm = tracks / np.array(tracks).sum(axis=1, keepdims=True)
y_pred_scaled = y_norm * quantity

In [33]:
# Divide individuals by genotype:

fasta = pyfastx.Fasta(os.path.join(SCRATCH, "example_tracks_and_deepshap/concat_sequence.fna.gz"))
seq_coords = [seq.name.split("_")[-1] for seq in fasta]

rs8050061_coord = "chr16:80231439-80232438"
rs8050061_seqs = [i for i in range(len(fasta)) if seq_coords[i] == rs8050061_coord]

c_pred = [y_pred_scaled[i, :] for i in rs8050061_seqs if fasta[i].seq[500] == "C"]
c_expt = [y[i, :] for i in rs8050061_seqs if fasta[i].seq[500] == "C"]
ct_pred = [y_pred_scaled[i, :] for i in rs8050061_seqs if fasta[i].seq[500] == "Y"]
ct_expt = [y[i, :] for i in rs8050061_seqs if fasta[i].seq[500] == "Y"]
t_pred = [y_pred_scaled[i, :] for i in rs8050061_seqs if fasta[i].seq[500] == "T"]
t_expt = [y[i, :] for i in rs8050061_seqs if fasta[i].seq[500] == "T"]

# Get mean per genotype

c_pred_mean = np.mean(np.array(c_pred), axis=0)
c_expt_mean = np.mean(np.array(c_expt), axis=0)
ct_pred_mean = np.mean(np.array(ct_pred), axis=0)
ct_expt_mean = np.mean(np.array(ct_expt), axis=0)
t_pred_mean = np.mean(np.array(t_pred), axis=0)
t_expt_mean = np.mean(np.array(t_expt), axis=0)

In [49]:
plot_side(
    c_pred_mean, ylim=[-6, 3], yticks=[0, 3],
    pic_name="img/model_fold_7_rs8050061C_pred.pdf"
)

In [48]:
plot_side(
    c_expt_mean, ylim=[-0.5, 0.25], yticks=[0, 0.25],
    pic_name="img/model_fold_7_rs8050061C_expt.pdf"
)

In [47]:
plot_side(
    t_pred_mean, ylim=[-6, 3], yticks=[0, 3],
    pic_name="img/model_fold_7_rs8050061T_pred.pdf"
)

In [46]:
plot_side(
    t_expt_mean, ylim=[-0.5, 0.25], yticks=[0, 0.25],
    pic_name="img/model_fold_7_rs8050061T_expt.pdf"
)

In [15]:
plot_side(ct_expt_mean, ylim=[-1, 0.5], yticks=[0, 0.5], pic_name="img/model_fold_7_rs8050061CT_expt.pdf")

In [16]:
plot_side(ct_pred_mean, ylim=[-6, 3], yticks=[0, 0.5], pic_name="img/model_fold_7_rs8050061CT_pred.pdf")

In [5]:
def l2_score(x, y, pseudocount=1e-5):
    return np.sqrt(np.sum(np.square(x - y), axis=-1))

l2_score(c_pred_mean, t_pred_mean)

4.4191

In [6]:
l2_score(c_expt_mean, t_expt_mean)

0.2861

In [52]:
c_pred_mean.sum()

12.593226469243358

In [53]:
c_expt_mean.sum()

1.357